<a href="https://colab.research.google.com/github/syedmahmoodiagents/GenAI/blob/main/Instruction_FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers datasets accelerate --q


In [ ]:
data = [
    {
        "instruction": "Translate to French",
        "input": "I love machine learning",
        "response": "J'aime l'apprentissage automatique"
    },
    {
        "instruction": "Explain briefly",
        "input": "What is an LLM?",
        "response": "An LLM is a large neural network trained on text to predict the next word."
    }
]


In [ ]:
def format_example(example):
    return f"""### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
{example['response']}"""


In [ ]:
import os, getpass

In [ ]:
os.environ['HF_TOKEN'] = getpass.getpass()

··········


In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def tokenize(example):
    text = format_example(example)
    tokens = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=256,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


In [ ]:
from datasets import Dataset

In [ ]:
dataset = Dataset.from_list(data)

In [ ]:
dataset.data

InMemoryTable
instruction: string
input: string
response: string
----
instruction: [["Translate to French","Explain briefly"]]
input: [["I love machine learning","What is an LLM?"]]
response: [["J'aime l'apprentissage automatique","An LLM is a large neural network trained on text to predict the next word."]]

In [ ]:
tokenized_dataset = dataset.map(tokenize, remove_columns=dataset.column_names)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments( per_device_train_batch_size=2, num_train_epochs=3, learning_rate=5e-5, report_to='none')

In [ ]:
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset)

In [ ]:
trainer.train()

In [ ]:
# training_args = TrainingArguments(
#     output_dir="./sft_output",
#     per_device_train_batch_size=2,
#     num_train_epochs=3,
#     learning_rate=5e-5,
#     logging_steps=1,
#     save_steps=50,
#     fp16=False,
# )


In [ ]:
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset)

In [ ]:
trainer.train()

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

# Testing..

In [ ]:
prompt = """### Instruction:
Translate to French

### Input:
I love deep learning

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
